In [ ]:
import requests
import geopandas as geo
import os
from datetime import datetime

Boundary data from geoBoundaries, Runfola, D. et al. (2020) geoBoundaries: A global database of political administrative boundaries. PLoS ONE 15(4): e0231866.

In [ ]:
# configuration

ISO_CODE = "RUS" #ISO 3166-1 alpha-3 code for Russia
ADM_LEVEL = "ADM1" # administrative level 1 for Kaliningrad Oblast
RELEASE_TYPE = "gbOpen" # open license
OUTPUT_DIR = r"C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\data\data"
OUTPUT_FILE = "kaliningrad-oblast.geojson"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# link for api access
url = f"https://www.geoboundaries.org/api/current/{RELEASE_TYPE}/{ISO_CODE}/{ADM_LEVEL}/"

try: 
    # fetch metadata
    response = requests.get(url)
    response.raise_for_status() # in case of error
    metadata = response.json()
    
    # downloading geojson
    geojson_url = metadata["gjDownloadURL"]
    print(f"Downloading GeoJSON from: {geojson_url}")
    
    geojson_response = requests.get(geojson_url)
    geojson_response.raise_for_status()
    
    # geopandas dataframe
    gdf = geo.read_file(geojson_response.text)
    
    # filter
    kaliningrad = gdf[gdf["shapeName"].str.contains("Kaliningrad", case=False, na=False)]
    
    if kaliningrad.empty:
        print("no features for Kaliningrad")
    else: 
        output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILE)
        kaliningrad.to_file(output_path, driver = "GeoJSON")
        print(f"saved Kaliningrad GeoJSON to: {output_path}")
        
        # elaborative info
        print("Kaliningrad Boundary Info:")
        print(f" - Number of features: {len(kaliningrad)}")
        print(f" - CRS: {kaliningrad.crs}")
        print(f" - Bounds: {kaliningrad.total_bounds}")

except requests.exceptions.RequestException as e:
    print(f"error during HTTP request: {e}")
except KeyError as e:
    print(f"error in metadata parsing: {e}")
except Exception as e:
    print(f"unexpected error: {e}")    

saved Kaliningrad GeoJSON to: C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\data\data\kaliningrad-oblast.geojson
Kaliningrad Boundary Info:
 - Number of features: 1
 - CRS: EPSG:4326
 - Bounds: [19.63892818 54.3182154  22.8866837  55.2936996 ]
